In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import tensorflow as tf
import os
from PIL import Image


In [ ]:
!rm -rf /content/color
!cp -r "/content/drive/MyDrive/Chatbot/PlantVillage/plantvillage dataset/color" /content/


In [ ]:
DATA_DIR = "/content/color"

SAVE_DIR = "/content/drive/MyDrive/Chatbot/plantClassificationModelsInEachEpoch"
os.makedirs(SAVE_DIR, exist_ok=True)


In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 20
VAL_SPLIT = 0.2
SEED = 42


In [ ]:
# ================================
# 2️⃣ REMOVE CORRUPTED / EMPTY FILES
# ================================
print("Checking corrupted files...")

bad_files = []

for root, dirs, files in os.walk(DATA_DIR):
    for file in files:
        path = os.path.join(root, file)
        try:
            if os.path.getsize(path) == 0:
                bad_files.append(path)
                continue
            img = Image.open(path)
            img.verify()
        except:
            bad_files.append(path)

for file in bad_files:
    os.remove(file)
print("Removed corrupted files:", len(bad_files))


Checking corrupted files...


In [ ]:
# ================================
# 3️⃣ LOAD DATASET
# ================================
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VAL_SPLIT,
    subset="training",
    seed=SEED,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VAL_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)


Found 54304 files belonging to 38 classes.
Using 43444 files for training.
Found 54304 files belonging to 38 classes.
Using 10860 files for validation.


In [ ]:
class_names = train_ds.class_names
num_classes = len(class_names)

print("Total Classes:", num_classes)


Total Classes: 38


In [ ]:
# ================================
# 4️⃣ NORMALIZATION
# ================================
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds   = val_ds.map(lambda x, y: (normalization_layer(x), y))

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds   = val_ds.prefetch(AUTOTUNE)


In [ ]:
# ================================
# 5️⃣ MODEL (EfficientNetB0)
# ================================
base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)



In [ ]:
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(num_classes, activation="softmax")
])


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(2e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)



In [ ]:
# ================================
# 6️⃣ CHECKPOINT CALLBACK
# ================================
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(SAVE_DIR, "epoch_{epoch:02d}.keras"),
    save_freq="epoch",
    save_best_only=False
)


In [ ]:
# ================================
# 7️⃣ TRAIN
# ================================
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[checkpoint_callback],
    verbose=1
)


Epoch 1/20
679/679 ━━━━━━━━━━━━━━━━━━━━ 118s 144ms/step - accuracy: 0.0883 - loss: 3.5601 - val_accuracy: 0.1133 - val_loss: 3.4747
Epoch 2/20
679/679 ━━━━━━━━━━━━━━━━━━━━ 107s 108ms/step - accuracy: 0.1128 - loss: 3.3899 - val_accuracy: 0.1017 - val_loss: 3.3822
Epoch 3/20
679/679 ━━━━━━━━━━━━━━━━━━━━ 67s 98ms/step - accuracy: 0.1273 - loss: 3.3390 - val_accuracy: 0.1017 - val_loss: 3.3541
Epoch 4/20
679/679 ━━━━━━━━━━━━━━━━━━━━ 66s 97ms/step - accuracy: 0.1373 - loss: 3.2989 - val_accuracy: 0.1017 - val_loss: 3.3467
Epoch 5/20
679/679 ━━━━━━━━━━━━━━━━━━━━ 66s 97ms/step - accuracy: 0.1508 - loss: 3.2468 - val_accuracy: 0.1017 - val_loss: 3.3947
Epoch 6/20
679/679 ━━━━━━━━━━━━━━━━━━━━ 66s 97ms/step - accuracy: 0.1529 - loss: 3.2082 - val_accuracy: 0.1017 - val_loss: 3.4886
Epoch 7/20
679/679 ━━━━━━━━━━━━━━━━━━━━ 82s 97ms/step - accuracy: 0.1595 - loss: 3.1723 - val_accuracy: 0.1018 - val_loss: 3.4682
Epoch 8/20
679/679 ━━━━━━━━━━━━━━━━━━━━ 66s 97ms/step - accuracy: 0.1650 - loss: 3.150

In [ ]:
# Save final model
model.save(os.path.join(SAVE_DIR, "final_model.keras"))


In [ ]:
import json
with open(os.path.join(SAVE_DIR, "class_names.json"), "w") as f:
    json.dump(class_names, f)

print("Training Completed.")


Training Completed.
